In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import Popen
import subprocess as sp
import os

import sys
sys.path.append('../modules/')

import IPython.display as ipd
import writeCmixSco_WT_ac as wRT_wt

# _ac stands for autocheck
cmixStatus, cmixResult = sp.getstatusoutput("CMIX")
if cmixStatus == 0:
    cmixInstalled = True
    print("CMIX found.")
else:
    cmixInstalled = False
    print("CMIX not found, using pyGoRTcmix instead.")


CMIX not found, using pyGoRTcmix instead.


In [2]:
cluster_sounds_dir = r"C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files"

os.listdir(cluster_sounds_dir)


['ayaya.wav',
 'bruh.wav',
 'kekw.wav',
 'monkaw_tos.wav',
 'nani_steve_harvey.wav',
 'No.wav',
 'REEEE.wav',
 'sarcastic_haha.wav',
 'senko-saying-poggers.wav',
 'yeet.wav']

In [3]:
import pandas as pd 
import random

def to_seconds(x):
    x = x.split(":")
    return int(x[0]) * 60**2 + int(x[1]) * 60 + float(x[2])

def vod2sound(csv_path,cluster_mappings,start_min,end_min,cluster_sounds_dir,sco_output_path):
    df = pd.read_csv(csv_path)
    df['sent_seconds'] = df['sent_time'].apply(to_seconds)
    df = df[(df['sent_seconds'] >= start_min*60) & (df['sent_seconds'] <= end_min*60)]
    df['sent_seconds'] -= df['sent_seconds'].iloc[0]
    print("{} rows from {} to {}".format(df.shape[0],start_min,end_min))
    
    sco_output_path = sco_output_path.format(start_min,end_min)
    f_out = open(sco_output_path , 'w')
    
    commands = """rtsetparams(44100,2)
load("SCRUB")
"""
    for cluster_idx in df['cluster_num'].unique():
        cluster_df = df[df['cluster_num']==cluster_idx]
        if cluster_mappings[cluster_idx] is None:
            continue 
        sound_path = os.path.join(cluster_sounds_dir,cluster_mappings[cluster_idx])
        if not os.path.exists(sound_path):
            print("WRONG SOUND INPUT PATH")
        commands += """rtinput("{}")\n""".format(sound_path)
        for i in range(cluster_df.shape[0]):
            start_sec = round(cluster_df.iloc[i]['sent_seconds'],2)
            amplitude = round(random.uniform(0.5,1.2),2)
            speed = round(random.uniform(1,2),2)
            left_perc = round(random.uniform(0,1),2)
            
            commands += "SCRUB({},0,4,{},{},16,40,0,{})\n".format(start_sec,amplitude,speed,left_perc)
    print(commands)
    f_out.write(commands)
    f_out.close()
    
    dur_sound = end_min*60 - start_min*60 +1 

    if cmixInstalled:
        cmix_cmd = 'CMIX < ' + base_name + '.sco'
        print(cmix_cmd)
    else:
        dur = str(dur_sound)
        cmix_cmd = os.path.abspath('../pyGoRTcmix/pyGoRTcmix') + ' -inputscore ' + os.path.abspath(sco_output_path) + ' -output ' + os.path.abspath(sco_output_path[:-4] + '.wav') + ' -dur ' + dur
        print(cmix_cmd)

    runCMIX = sp.Popen(cmix_cmd, shell=True) # if can only be called from a shell, use shell=True

    runCMIX.wait()


In [9]:
CLUSTER_MAPPING_RIOT = {
    0: "senko-saying-poggers.wav",
    1: "ayaya.wav",
    2: "kekw.wav",
    3: "nani_steve_harvey.wav",
    4: "yeet.wav",
    5: "monkaw_tos.wav",
    6: None,
    7: "REEEE.wav",
    8: None,
    9: "No.wav"
}

vod2sound(
    r"../../data/cluster_outputs/riot_games_LoL\v2\756260657.csv",
    cluster_mappings=CLUSTER_MAPPING_RIOT,
    start_min=45,
    end_min=46,
    cluster_sounds_dir=cluster_sounds_dir,
    sco_output_path= r"../cluster_sounds/output/riot_games_756260657_{}_{}.sco"
)

56 rows from 45 to 46
rtsetparams(44100,2)
load("SCRUB")
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\monkaw_tos.wav")
SCRUB(0.0,0,4,0.82,1.24,16,40,0,0.51)
SCRUB(16.66,0,4,0.8,1.03,16,40,0,0.93)
SCRUB(19.2,0,4,1.06,1.64,16,40,0,0.54)
SCRUB(40.95,0,4,0.86,1.1,16,40,0,0.58)
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\yeet.wav")
SCRUB(0.31,0,4,0.93,1.08,16,40,0,0.04)
SCRUB(0.33,0,4,1.12,1.59,16,40,0,0.54)
SCRUB(1.35,0,4,1.05,1.5,16,40,0,0.93)
SCRUB(4.82,0,4,1.05,1.78,16,40,0,0.49)
SCRUB(7.32,0,4,1.03,1.05,16,40,0,0.99)
SCRUB(7.85,0,4,0.79,1.36,16,40,0,0.17)
SCRUB(7.94,0,4,0.63,1.8,16,40,0,0.75)
SCRUB(8.66,0,4,0.81,1.17,16,40,0,0.67)
SCRUB(9.26,0,4,1.13,1.77,16,40,0,0.81)
SCRUB(11.04,0,4,0.7,1.71,16,40,0,0.29)
SCRUB(15.46,0,4,0.63,1.53,16,40,0,0.26)
SCRUB(18.03,0,4,0.76,1.12,16,40,0,0.45)
SCRUB(20.9,0,4,0.66,1.9,16,40,0,0.15)
SCRUB(23.74,0,4,1.1,1.49,16,40,0,0.51)
SCRUB(25.73,0,4,0

In [5]:
CLUSTER_MAPPING_RIOT = {
    0: "senko-saying-poggers.wav",
    1: "ayaya.wav",
    2: "kekw.wav",
    3: "nani_steve_harvey.wav",
    4: "yeet.wav",
    5: "monkaw_tos.wav",
    6: None,
#     7: "REEEE.wav",
    7: "yeet.wav",
    8: None,
    9: "No.wav"
}

vod2sound(
    r"../../data/cluster_outputs/riot_games_LoL\v2\756260657.csv",
    cluster_mappings=CLUSTER_MAPPING_RIOT,
    start_min=156,
    end_min=157,
    cluster_sounds_dir=cluster_sounds_dir,
    sco_output_path= r"../cluster_sounds/output/riot_games_756260657_{}_{}.sco"
)

361 rows from 156 to 157
rtsetparams(44100,2)
load("SCRUB")
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\No.wav")
SCRUB(0.0,0,4,0.62,1.43,16,40,0,0.25)
SCRUB(4.16,0,4,0.97,1.4,16,40,0,0.87)
SCRUB(7.63,0,4,0.85,1.88,16,40,0,0.25)
SCRUB(11.12,0,4,0.99,1.76,16,40,0,0.11)
SCRUB(16.85,0,4,0.85,1.45,16,40,0,0.28)
SCRUB(18.84,0,4,1.19,1.69,16,40,0,0.32)
SCRUB(21.93,0,4,0.91,1.81,16,40,0,0.41)
SCRUB(28.33,0,4,0.53,1.32,16,40,0,0.07)
SCRUB(40.16,0,4,0.67,1.07,16,40,0,0.83)
SCRUB(49.6,0,4,1.04,1.52,16,40,0,0.03)
SCRUB(58.84,0,4,0.51,1.52,16,40,0,0.55)
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\ayaya.wav")
SCRUB(0.43,0,4,1.02,1.56,16,40,0,0.8)
SCRUB(0.55,0,4,0.94,1.25,16,40,0,0.13)
SCRUB(0.65,0,4,1.01,1.25,16,40,0,0.09)
SCRUB(0.9,0,4,1.18,1.6,16,40,0,0.82)
SCRUB(0.95,0,4,0.89,1.83,16,40,0,0.08)
SCRUB(2.23,0,4,1.08,1.91,16,40,0,0.29)
SCRUB(2.32,0,4,1.09,1.67,16,40,0,0.93)
SCRUB(2.35,0,4,0.

In [11]:
CLUSTER_MAPPING_XQC = {
    0: "senko-saying-poggers.wav",
    1: None,
    2: "kekw.wav",
    3: "nani_steve_harvey.wav",
    4: "yeet.wav",
    5: "monkaw_tos.wav",
    6: None,
    7: "ayaya.wav",
    8: None,
    9: "No.wav"
}

vod2sound(
    r"../../data/cluster_outputs/xQcOW_minecraft/v2/893334860.csv",
    cluster_mappings=CLUSTER_MAPPING_XQC,
    start_min=30,
    end_min=31,
    cluster_sounds_dir=cluster_sounds_dir,
    sco_output_path= r"../cluster_sounds/output/xQcOW_893334860_{}_{}.sco"
)

486 rows from 30 to 31
rtsetparams(44100,2)
load("SCRUB")
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\kekw.wav")
SCRUB(0.0,0,4,1.18,1.0,16,40,0,0.29)
SCRUB(0.75,0,4,0.94,1.44,16,40,0,0.78)
SCRUB(6.52,0,4,0.77,1.02,16,40,0,0.12)
SCRUB(9.22,0,4,0.9,1.49,16,40,0,0.19)
SCRUB(9.45,0,4,1.16,1.98,16,40,0,0.83)
SCRUB(52.16,0,4,0.73,1.53,16,40,0,0.93)
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\monkaw_tos.wav")
SCRUB(0.08,0,4,1.19,1.75,16,40,0,0.2)
SCRUB(0.55,0,4,0.76,1.77,16,40,0,0.58)
SCRUB(0.77,0,4,1.14,1.04,16,40,0,0.55)
SCRUB(3.21,0,4,0.9,1.71,16,40,0,0.4)
SCRUB(3.79,0,4,0.67,1.35,16,40,0,0.43)
SCRUB(3.8,0,4,0.91,1.79,16,40,0,0.88)
SCRUB(6.86,0,4,0.81,1.64,16,40,0,0.51)
SCRUB(9.11,0,4,0.81,1.73,16,40,0,0.46)
SCRUB(9.54,0,4,0.68,1.53,16,40,0,0.08)
SCRUB(9.59,0,4,0.91,1.78,16,40,0,0.5)
SCRUB(9.68,0,4,0.53,1.26,16,40,0,0.54)
SCRUB(9.76,0,4,0.79,1.53,16,40,0,0.76)
SCRUB(12.81,0,4,0.69,

In [32]:
# play RTcmix score and create a wave file
# use CMIX if installed; if not, use pyGoRTcmix
abitextra = 0.5
dur_sound = 5 + abitextra 

if cmixInstalled:
    cmix_cmd = 'CMIX < ' + base_name + '.sco'
    print(cmix_cmd)
else:
    dur = str(dur_sound)
    cmix_cmd = os.path.abspath('../pyGoRTcmix/pyGoRTcmix') + ' -inputscore ' + os.path.abspath(score_file_name) + ' -output ' + os.path.abspath(score_file_name[:-4] + '.wav') + ' -dur ' + dur
    print(cmix_cmd)

runCMIX = sp.Popen(cmix_cmd, shell=True) # if can only be called from a shell, use shell=True

runCMIX.wait()


C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\pyGoRTcmix\pyGoRTcmix -inputscore C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\notebooks\test.sco -output C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\notebooks\test.wav -dur 5.5


0

In [12]:
CLUSTER_MAPPING_SMASH = {
    0: "senko-saying-poggers.wav",
    1: "REEEE.wav",
    2: "kekw.wav",
    3: "nani_steve_harvey.wav",
    4: "yeet.wav",
    5: "monkaw_tos.wav",
    6: None,
    7: "ayaya.wav",
    8: None,
    9: "No.wav"
}

vod2sound(
    r"../../data/cluster_outputs/smash_tournaments/v2/374037944.csv",
    cluster_mappings=CLUSTER_MAPPING_XQC,
    start_min=45*5,
    end_min=45*5+1,
    cluster_sounds_dir=cluster_sounds_dir,
    sco_output_path= r"../cluster_sounds/output/smash_tournaments_374037944_{}_{}.sco"
)

89 rows from 225 to 226
rtsetparams(44100,2)
load("SCRUB")
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\yeet.wav")
SCRUB(0.0,0,4,0.76,1.91,16,40,0,0.86)
SCRUB(19.77,0,4,1.02,1.35,16,40,0,0.06)
SCRUB(32.53,0,4,1.13,1.88,16,40,0,0.07)
SCRUB(32.64,0,4,0.68,1.72,16,40,0,0.52)
SCRUB(37.37,0,4,0.86,1.62,16,40,0,0.71)
SCRUB(39.18,0,4,0.88,1.38,16,40,0,0.79)
SCRUB(49.84,0,4,0.7,1.23,16,40,0,0.85)
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\monkaw_tos.wav")
SCRUB(4.09,0,4,0.81,1.86,16,40,0,0.01)
SCRUB(5.03,0,4,0.68,1.33,16,40,0,0.11)
SCRUB(7.76,0,4,1.09,1.4,16,40,0,0.93)
SCRUB(12.22,0,4,0.89,1.91,16,40,0,0.36)
SCRUB(19.29,0,4,0.89,1.96,16,40,0,0.34)
SCRUB(22.02,0,4,1.12,1.47,16,40,0,0.06)
SCRUB(28.79,0,4,0.83,1.62,16,40,0,0.79)
SCRUB(29.3,0,4,0.93,1.73,16,40,0,0.81)
SCRUB(39.19,0,4,1.07,1.16,16,40,0,0.61)
SCRUB(39.81,0,4,1.11,1.21,16,40,0,0.86)
SCRUB(45.0,0,4,0.72,1.74,16,40,0,0.42)
SCRU

In [31]:
score_file_name = "test.sco"

f_out = open(score_file_name , 'w')


# commands = r"""
# rtsetparams(44100, 2)
# fname = "C:\Users\Shoya\columbia\musi6602\datamovies_y21\project\cluster_sounds\wav_files\sarcastic_haha.wav"
# rtinput(fname)
# ampenv = maketable("line", 1000, 0,0, 1,1)

# MIX(0, 0, 7.0, 1*ampenv, 0, 0)
# MIX(0.3, 0, 7.0, 1*ampenv, 0, 0)
# """

"""
p0 = output start time (seconds)
   p1 = input start time (seconds)
   p2 = output duration (or endtime if negative) (seconds)
   p3 = amplitude multiplier (relative multiplier of input signal)
   p4 = speed multiplier
   p5 = sync width (samples)
   p6 = sync oversampling (samples)
   p7 = input channel [optional; default is 0]
   p8 = percent to left [optional; default is .5]
"""
"""
['ayaya.wav',
 'bruh.wav',
 'kekw.wav',
 'monkaw_tos.wav',
 'nani_steve_harvey.wav',
 'No.wav',
 'REEEE.wav',
 'sarcastic_haha.wav',
 'senko-saying-poggers.wav',
 'yeet.wav']
"""


commands = r"""
rtsetparams(44100,2)
load("SCRUB")

rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\yeet.wav")

speed = 1.5
dur = 1
skip = 0
// Play forward from time 0
SCRUB(0, skip, dur, 1, speed,16,40,0,0.5)
"""

commands = r"""
rtsetparams(44100,2)
load("SCRUB")
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\kekw.wav")
SCRUB(0.0,0,4,0.91,1.85,16,40,0,0.01)
"""

# commands = r"""
# rtsetparams(44100,2)
# load("SCRUB")

# fname1 = "C:\Users\Shoya\columbia\musi6602\datamovies_y21\project\cluster_sounds\wav_files\sarcastic_haha.wav"
# rtinput(fname1)
# speed = 1.5
# dur = 1
# skip = 0
# // Play forward from time 0
# SCRUB(0, skip, dur, 1, speed, 16, 80,0,0)


# fname2 ="C:\Users\Shoya\columbia\musi6602\datamovies_y21\project\cluster_sounds\wav_files\yeet.wav"
# rtinput(fname2)

# speed=1
# dur=2
# skip = 0
# // Play forward from time 0
# SCRUB(0.5, skip, dur, 1, speed, 16, 40,1,1)

# """

# fname2 = "C:\Users\Shoya\columbia\musi6602\datamovies_y21\project\cluster_sounds\wav_files\scream.wav"
print(commands)
f_out.write(commands)
f_out.close()


rtsetparams(44100,2)
load("SCRUB")
rtinput("C:\Users\Shoya\columbia\musi6602\TwitchSonification\sonification\cluster_sounds\wav_files\kekw.wav")
SCRUB(0.0,0,4,0.91,1.85,16,40,0,0.01)

